### Creating Catalog, Schema and Table

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ctl_joecrypto;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_joecrypto.staging_schema;

In [0]:
gold_path = "abfss://gold@joecryptostorage01.dfs.core.windows.net/"
silver_path = "abfss://silver@joecryptostorage01.dfs.core.windows.net/silver_crypto"

In [0]:
%sql
SHOW EXTERNAL LOCATIONS

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_joecrypto.staging_schema.silvercryptofinal
USING DELTA
LOCATION "abfss://silver@joecryptostorage01.dfs.core.windows.net/silver_crypto";

In [0]:
%sql
SELECT * FROM ctl_joecrypto.staging_schema.silvercryptofinal 
LIMIT 5;

### Normalizing the General Table - silvercryptofinal
-- By extracting the fact and dimension tables

In [0]:
%sql
-- Creating a seperate schema for gold tables
CREATE SCHEMA IF NOT EXISTS ctl_joecrypto.gold_schema;

In [0]:
%sql
-- DROP TABLE IF EXISTS ctl_joecrypto.staging_schema.dim_roi
-- delete the dropped table from the storage too;
-- DROP TABLE IF EXISTS ctl_joecrypto.gold_schema.fact_crypto


In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_joecrypto.gold_schema.dim_roi(
  roi_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING,
  roi_time DOUBLE,
  roi_currency STRING,
  roi_percentage DOUBLE,
  last_updated_ts TIMESTAMP
  )
  USING DELTA
  LOCATION "abfss://gold@joecryptostorage01.dfs.core.windows.net/dim_roi";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_joecrypto.gold_schema.dim_crypto
(
  dim_crypto_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING NOT NULL,
  name STRING,
  symbol STRING,
  ath DOUBLE,
  ath_change_percentage DOUBLE,
  market_cap_billions DOUBLE,
  volume_to_market_cap_ratio DOUBLE,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION "abfss://gold@joecryptostorage01.dfs.core.windows.net/dim_crypto"

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_joecrypto.gold_schema.fact_crypto
(
  id STRING PRIMARY KEY,
  market_cap_rank BIGINT,
  max_supply DOUBLE,
  total_supply DOUBLE,
  circulating_supply DOUBLE,
  total_volume DOUBLE,
  market_cap DOUBLE,
  market_cap_change_24h DOUBLE, 
  market_cap_change_percentage_24h DOUBLE,
  price_change_24h DOUBLE,
  price_change_percentage_24h DOUBLE,
  fully_diluted_valuation DOUBLE,
  market_cap_billions DOUBLE,
  high_24h DOUBLE,
  low_24h DOUBLE,
  current_price DOUBLE,
  date_partition DATE, 
  last_updated_ts TIMESTAMP

)
USING DELTA
LOCATION "abfss://gold@joecryptostorage01.dfs.core.windows.net/fact_crypto";

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_joecrypto.gold_schema.dim_price_history (
  price_history_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING NOT NULL,
  ath DOUBLE,
  ath_change_percentage DOUBLE,
  ath_date_ts TIMESTAMP,
  atl DOUBLE,
  atl_change_percentage DOUBLE,
  atl_date_ts TIMESTAMP,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION "abfss://gold@joecryptostorage01.dfs.core.windows.net/dim_price_history";

In [0]:
%sql
MERGE INTO ctl_joecrypto.gold_schema.dim_roi AS target
USING (
  SELECT
    id,
    roi_time,
    roi_currency,
    roi_percentage,
    last_updated_ts,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (
    SELECT *, 
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn 
    FROM ctl_joecrypto.staging_schema.silvercryptofinal
  ) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.roi_time = updated_rn.roi_time,
    target.roi_currency = updated_rn.roi_currency,
    target.roi_percentage = updated_rn.roi_percentage,
    target.last_updated_ts = updated_rn.last_updated_ts

WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id, 
    roi_time,
    roi_currency,
    roi_percentage,
    last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.roi_time,
    updated_rn.roi_currency,
    updated_rn.roi_percentage, 
    updated_rn.last_updated_ts    
    
  );

In [0]:
%sql
MERGE INTO ctl_joecrypto.gold_schema.dim_crypto AS target
USING (
  SELECT
    id,
    name,
    symbol,
    ath,
    ath_change_percentage,
    market_cap_billions,
    volume_to_market_cap_ratio,
    last_updated_ts,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn FROM ctl_joecrypto.staging_schema.silvercryptofinal) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.name = updated_rn.name,
    target.symbol = updated_rn.symbol,
    target.ath = updated_rn.ath,
    target.ath_change_percentage = updated_rn.ath_change_percentage,
    target.market_cap_billions = updated_rn.market_cap_billions,
    target.volume_to_market_cap_ratio = updated_rn.volume_to_market_cap_ratio,
    target.last_updated_ts = updated_rn.last_updated_ts
    WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id, 
    name,
    symbol,
    ath,
    ath_change_percentage,
    market_cap_billions,
    volume_to_market_cap_ratio,
    last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.name,
    updated_rn.symbol,
    updated_rn.ath,
    updated_rn.ath_change_percentage,
    updated_rn.market_cap_billions,
    updated_rn.volume_to_market_cap_ratio,
    updated_rn.last_updated_ts
  );

In [0]:
%sql
MERGE INTO ctl_joecrypto.gold_schema.dim_price_history AS target
USING (
  SELECT
   id,
   ath,
   ath_change_percentage,
   ath_date_ts,
   atl,
   atl_change_percentage,
   atl_date_ts,
   last_updated_ts,
   ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn FROM ctl_joecrypto.staging_schema.silvercryptofinal) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.ath = updated_rn.ath,
    target.ath_change_percentage = updated_rn.ath_change_percentage,
    target.ath_date_ts = updated_rn.ath_date_ts,
    target.atl = updated_rn.atl,
    target.atl_change_percentage = updated_rn.atl_change_percentage,
    target.atl_date_ts = updated_rn.atl_date_ts,
    target.last_updated_ts = updated_rn.last_updated_ts
   
    WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id,
   ath,
   ath_change_percentage,
   ath_date_ts,
   atl,
   atl_change_percentage,
   atl_date_ts,
   last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.ath,
    updated_rn.ath_change_percentage,
    updated_rn.ath_date_ts,
    updated_rn.atl,
    updated_rn.atl_change_percentage,
    updated_rn.atl_date_ts,
    updated_rn.last_updated_ts
  );
    

In [0]:
%sql
MERGE INTO ctl_joecrypto.gold_schema.fact_crypto AS target
USING (
  SELECT 
    id,
    market_cap_rank,
    max_supply,
    total_supply,
    circulating_supply,
    total_volume,
    market_cap,
    market_cap_change_24h,
    market_cap_change_percentage_24h,
    price_change_24h,
    price_change_percentage_24h,
    fully_diluted_valuation,
    market_cap_billions,
    high_24h,
    low_24h,
    current_price,
    date_partition, 
    last_updated_ts,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY id ORDER BY last_updated_ts DESC) AS rn FROM ctl_joecrypto.staging_schema.silvercryptofinal) WHERE rn = 1
) AS updated_rn

ON target.id = updated_rn.id

WHEN MATCHED AND updated_rn.rn = 1 THEN 
  UPDATE SET
    target.market_cap_rank = updated_rn.market_cap_rank,
    target.max_supply = updated_rn.max_supply,
    target.total_supply = updated_rn.total_supply,
    target.circulating_supply = updated_rn.circulating_supply,
    target.total_volume = updated_rn.total_volume,
    target.market_cap = updated_rn.market_cap,
    target.market_cap_change_24h = updated_rn.market_cap_change_24h,
    target.market_cap_change_percentage_24h = updated_rn.market_cap_change_percentage_24h,
    target.price_change_24h = updated_rn.price_change_24h,
    target.price_change_percentage_24h = updated_rn.price_change_percentage_24h,
    target.fully_diluted_valuation = updated_rn.fully_diluted_valuation,
    target.market_cap_billions = updated_rn.market_cap_billions,
    target.high_24h = updated_rn.high_24h,
    target.low_24h = updated_rn.low_24h,
    target.current_price = updated_rn.current_price,
    target.date_partition = updated_rn.date_partition,  
    target.last_updated_ts = updated_rn.last_updated_ts

WHEN NOT MATCHED AND updated_rn.rn = 1 THEN 
  INSERT (
    id,
    market_cap_rank,
    max_supply,
    total_supply,
    circulating_supply,
    total_volume,
    market_cap,
    market_cap_change_24h,
    market_cap_change_percentage_24h,
    price_change_24h,
    price_change_percentage_24h,
    fully_diluted_valuation,
    market_cap_billions,
    high_24h,
    low_24h,
    current_price,
    date_partition,  
    last_updated_ts
  ) 
  VALUES (
    updated_rn.id,
    updated_rn.market_cap_rank,
    updated_rn.max_supply,
    updated_rn.total_supply,
    updated_rn.circulating_supply,
    updated_rn.total_volume,
    updated_rn.market_cap,
    updated_rn.market_cap_change_24h,
    updated_rn.market_cap_change_percentage_24h,
    updated_rn.price_change_24h,
    updated_rn.price_change_percentage_24h,
    updated_rn.fully_diluted_valuation,
    updated_rn.market_cap_billions,
    updated_rn.high_24h,
    updated_rn.low_24h,
    updated_rn.current_price,
    updated_rn.date_partition,  
    updated_rn.last_updated_ts
  );

In [0]:
transformed_data = spark.sql("""
SELECT
    dc.dim_crypto_id,
    cf.id,
    name,
    symbol,
    ath,
    ath_change_percentage,
    cf.market_cap,
    volume_to_market_cap_ratio,
    (price_change_24h / cf.market_cap_billions) AS normalized_price_change_24h
FROM
    ctl_joecrypto.gold_schema.fact_crypto cf
JOIN
    ctl_joecrypto.gold_schema.dim_crypto dc
ON cf.id = dc.id
""")